# Exercise 7

I recommend to run this notebook on google colab or on some other computer with access to a GPU. Otherwise, the fine-tuning will take very long and you cannot compare the runtime of GPU and CPU versions. 

In [ ]:
import os

IS_ON_COLAB = bool(os.getenv("COLAB_RELEASE_TAG"))

if IS_ON_COLAB:
    !pip install transformers==4.28.0
    !pip install tokenizers datasets sentencepiece huggingface_hub[cli] accelerate

In [ ]:
from transformers import pipeline
from datasets import load_dataset
from transformers import logging
import torch
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report
from time import time
from transformers import AutoTokenizer
import scipy

logging.set_verbosity_error()

## Task 1: Create a text classification pipeline

The following task uses concepts from lecture 3 (working with pipeline). Revisit the lecture slides if you get stuck.

1. create a classifier using the `pipeline` function from transformers and the model `"jsoutherland/distilbert-base-uncased-finetuned-emotion"`.
2. Use the model to classify the validation-split of the emotion dataset
3. Write down how long it took to do the classification

In [ ]:
emotions = load_dataset("dair-ai/emotion", name="split")

## Task 2: Calculate sklearn scores for huggingface models

1. Translate the predictions into numerical values, using the following encoding: "sadness": 0, "joy": 1, "love": 2, "anger": 3, "fear": 4,"surprise": 5.
2. Create a classification report by comparing the predictions against the true labels from the validation dataset. 
3. If you have time left, create a confusion matrix or other scores

## Task 3: Compare CPU and GPU speed

0. Go to Edit -> Notebook Settings and select a GPU accelerator
1. Create another random matrix of the same shape as a that lives on the gpu (if you have one)
2. Measure the runtime of multiplying a with itself
3. Measure the runtime of multiplying the gpu version with itself
4. Calculate how much faster the GPU version was
5. If you have time left, try different matrix sizes, especially tiny matrices

**Note:** For me, using `%timeit` did not work for the GPU version on google colab. Use `time` instead. 

In [ ]:
a = torch.randn(5_000, 5_000)

## Task 4: Pipeline with GPU

1. Create a new pipeline with `device="cuda:0"` and re-run the classification
2. Compare the runtimes

## Task 5: Loading a model

1. Load a distilbert-base-uncased model for sequence classification that can be fine-tuned on the emotions dataset. Put it on the gpu. 

In [ ]:
model_name = "distilbert-base-uncased"

tokenizer = AutoTokenizer.from_pretrained(model_name)


def tokenize(batch):
    return tokenizer(batch["text"], padding=True, truncation=True)


emotions_encoded = emotions.map(tokenize, batched=True, batch_size=None)
emotions_encoded.set_format(
    "torch",
    columns=["input_ids", "attention_mask", "label"],
)
emotions_encoded.set_format("torch")
emotions_encoded

## Task 6: Specifying the training options and training the model

1. Write a compute_metrics function or copy paste it from the slides
2. Specify training arguments. Choose the same values as in the slides but only 1 epoch to make it faster.
3. Create a Trainer instance
4. Train the model

## Task 7: Use the fine tuned model

1. Tokenize the text and run it through the model to get logits
2. Use scipy.special.softmax to create probabilities
3. Plot the probabilities of each class (optional)

In [ ]:
custom_text = "I am glad the class is almost over."